<a href="https://colab.research.google.com/github/RC11-SkillsClass2022-23/KexinGao_Augustine/blob/main/Segment_for_highlighting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Auto-generating mask

##install

In [1]:
!pip install git+https://github.com/facebookresearch/segment-anything.git

import os
import cv2
import json
import numpy as np
from PIL import Image
from segment_anything.utils.onnx import SamOnnxModel
from segment_anything import sam_model_registry, SamPredictor, SamAutomaticMaskGenerator
sam = sam_model_registry["vit_h"](checkpoint="/content/drive/MyDrive/for_masks/sam_vit_h_4b8939.pth")
sam.to(device = "cuda")
mask_generator = SamAutomaticMaskGenerator(sam)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-of5ojlk9
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-of5ojlk9
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36610 sha256=f744a88a4a2d0fd803dfd718f0c954c8f94120efbafcd5d4622eae43cfb4970b
  Stored in directory: /tmp/pip-ephem-wheel-cache-aaivv0yw/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment-anything


##One image

In [2]:
image = '/content/drive/MyDrive/for_masks/demo/output_229.jpg'
img = cv2.imread(image)
masks = mask_generator.generate(img) #the generated mask
masks

[{'segmentation': array([[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False]]),
  'area': 21158,
  'bbox': [1043, 409, 108, 274],
  'predicted_iou': 1.0189889669418335,
  'point_coords': [[1062.0, 573.75]],
  'stability_score': 0.9830834269523621,
  'crop_box': [0, 0, 1152, 720]},
 {'segmentation': array([[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False]]),
  'area': 13763,
  'bbox': [1043, 459,

In [ ]:
#masks is a dictionary with several properties
#segmentation in mask is an ndarray, can be packed with .npy
#export each mask packed in a single .npy file

n = 1
for i in masks:
  name = i['segmentation']
  output_path = '/content/drive/MyDrive/for_masks/save/01/01_'+str(n)+'.npy'#stores only one array in one .npy
  np.save(output_path, name)
  n = n+1

In [ ]:
#for visualization, transform the array into png images
array = np.load('/content/drive/MyDrive/for_masks/save/01/01_1.npy')
image = Image.fromarray(array)
image.save('/content/drive/MyDrive/for_masks/save/01/01_1.png')

##is there a need for packing masks in one file?

In [ ]:
#is there a need for packing masks in one file?
np.savez('/content/drive/MyDrive/for_masks/data.npz',)#names of all the arrays need packing

##Bach processing

In [14]:
def Segmentation(folder):
  #load the images
  a = 1
  image_list = os.listdir(folder)
  images = []
  for image in image_list:
    images.append(os.path.join(folder, image))
  for image in images:
    img = cv2.imread(image)
    masks = mask_generator.generate(img)
    #download the masks
    n = 1
    for i in masks:
      name = i['segmentation']
      output_path = '/content/drive/MyDrive/for_masks/save/'+str(a)+'/'+str(a)+'_'+str(n)+'.npy'
      if n == 1:
        output_folder = '/content/drive/MyDrive/for_masks/save/'+str(a)      
        os.makedirs(output_folder)
      np.save(output_path, name)
      n = n+1
    a = a+1

In [15]:
#input a folder of frames
folder = '/content/drive/MyDrive/for_masks/demo'
Segmentation(folder)